In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%store -r student_courses
%store -r student_majors
%store -r stem_paths
%store -r maker_passes
%store -r jacobs

In [3]:
student_majors.query('ppsk == 6417')

,ppsk,undergrad.status,year.majors,term.majors,Count of Students,college,division,department,major,exam.units
0,6417,U,2013,Fall,1,Clg of Letters & Science,L&S Social Sciences Division,Political Science,Political Science,0.0
1,6417,U,2014,Spring,1,Clg of Letters & Science,L&S Social Sciences Division,Political Science,Political Science,0.0


In [10]:
jacobs['ppsk']

9995     1474600
12946    2631969
15727     723198
16929     917618
18297    1025236
          ...   
21309    4262284
21441    4262621
22125    4264306
22572    4265873
22697    4266391
Name: ppsk, Length: 2255, dtype: int64

In [5]:
#temp = student_courses[student_courses['year.name'].str.split(" ", expand=True)[0].astype(int) >= 2015]

In [6]:
# ppsks of everyone who (in data) uses Jacobs for something
ppsks = maker_passes['ppsk']
ppsks = ppsks.append(jacobs['ppsk']).unique()

# all Jacobs students' academic career, either with DESINV course or MakerPass
all_jacobs = student_courses[(student_courses['ppsk'].isin(ppsks)) & (student_courses['undergrad.status'] == 'Undergraduate')]

In [7]:
#test = all_jacobs.groupby(['ppsk','year.name']).filter(lambda x: (x['course.subject'].isin(['Design Innovation']).any()) & (~x['student.credit.hours'].isna().any()))
#graph_data = test.groupby(['ppsk','year.name'])[['student.credit.hours']].sum()

In [26]:
all_jacobs

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,instructor.name,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
3183,2007 Fall,724096,Undergraduate,Business,Business Admin-Undergrad,103,Primary,1,8553,8553.0,Intro To Finance,Jonathan Berk,A,Letter Grade,A+,4.0,4.0
3184,2007 Fall,724096,Undergraduate,Psychology,Psychology,141,Primary,1,74426,74426.0,Devel Dur Infancy,Joseph Campos,A,Letter Grade,A+,4.0,3.0
3185,2007 Fall,724096,Undergraduate,Psychology,Psychology,150,Primary,1,74453,74453.0,Personality Psych,Meg Jay,A,Letter Grade,A+,4.0,3.0
4065,2007 Fall,765357,Undergraduate,Integrative Biology,Integrative Biology,88,Primary,1,43020,43020.0,Leaders Bio Scholar,Alissa Myrick; Katherine Kim; Tyrone Hayes,A,Letter Grade,A+,4.0,1.0
8987,2007 Summer,713359,Undergraduate,Electrical Eng & Computer Sci,Electrical Engineering,100,Primary,1,37285,200007.0,Elec Tech For Eng,Bharathwaj Muthuswamy; Jean Walrand,A,Letter Grade,A,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22126,2019 Summer,4264306,Undergraduate,Electrical Eng & Computer Sci,Computer Science,C8,Primary,1,14720,25.0,Foundation Data Sci,NaN,A,Letter Grade,A,4.0,4.0
22531,2019 Summer,4265730,Undergraduate,Electrical Eng & Computer Sci,Computer Science,61A,Primary,1,14622,14622.0,Str Interp Cmp Prgs,NaN,A,Letter Grade,A,4.0,4.0
22572,2019 Summer,4265873,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,NaN,B,Letter Grade,B,3.0,2.0
22573,2019 Summer,4265873,Undergraduate,Gender & Womens Studies,Gender & Womens Studies,111,Primary,2,14306,14306.0,Special Topics,NaN,A,Letter Grade,A,4.0,4.0


In [49]:
student_majors

,ppsk,undergrad.status,year.name,Count of Students,college,division,department,major,exam.units
0,193666,U,2020 Spring,1,Clg of Letters & Science,L&S Math & Phys Sciences Div,Physics,Physics,NaN
1,244810,U,2020 Spring,1,Clg of Letters & Science,L&S Social Sciences Division,History,History,NaN
2,392960,U,2020 Spring,1,Clg of Letters & Science,L&S Social Sciences Division,Economics,Economics,NaN
3,384263,U,2020 Spring,1,Clg of Letters & Science,L&S Social Sciences Division,History,History,NaN
4,373868,U,2020 Spring,1,Clg of Letters & Science,L&S Undergrad Studies Division,UG Interdisciplinary Studies,American Studies,NaN
...,...,...,...,...,...,...,...,...,...
65494,651209,U,2007 Spring,1,Clg of Letters & Science,L&S Undergrad Studies Division,UG Interdisciplinary Studies,Mass Communications,10.7
65495,651209,U,2007 Spring,1,Haas School of Business,Haas School of Business,Business,Business Administration,10.7
65496,651209,U,2008 Spring,1,Clg of Letters & Science,L&S Undergrad Studies Division,UG Interdisciplinary Studies,Mass Communications,10.7
65497,651209,U,2008 Spring,1,Haas School of Business,Haas School of Business,Business,Business Administration,10.7


In [15]:
all_jacobs.groupby(['ppsk','year.name']).filter(lambda x: (x['course.subject'].isin(['Design Innovation']).any()) 
single_students = all_jacobs.groupby('ppsk')

SyntaxError: invalid syntax (<ipython-input-15-dbc2242fb19f>, line 2)

In [9]:
test.query('ppsk == 1029128')

NameError: name 'test' is not defined

In [37]:
test.query('ppsk == 1029128')['student.credit.hours'].sum()

9.5

In [11]:
#get ppsks of students from jacobs and find other classes they take
# how many units students take
all_jacobs = student_courses

In [12]:
jacobs

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
9995,2015 Fall,1474600,Undergraduate,Engineering Science,Design Innovation,90,Primary,1,18400,101548.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,2.0
12946,2015 Fall,2631969,Undergraduate,Engineering Science,Design Innovation,90,Primary,1,18400,101548.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,2.0
15727,2015 Spring,723198,Graduate,Engineering Science,Design Innovation,190,Primary,2,18406,18406.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0
16929,2015 Spring,917618,Undergraduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0
18297,2015 Spring,1025236,Graduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21309,2019 Summer,4262284,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,A,Letter Grade,A-,3.7,2.0
21441,2019 Summer,4262621,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,A,Letter Grade,A-,3.7,2.0
22125,2019 Summer,4264306,Undergraduate,Design Innovation,Design Innovation,190,Primary,8,15779,15779.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,3.0
22572,2019 Summer,4265873,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,B,Letter Grade,B,3.0,2.0


In [13]:
jacobs_ethnicities = jacobs.merge(stem_paths[['ppsk', 'ethnic.l1']], on='ppsk', how='left')
jacobs_ethnicities['ethnic.l1'].fillna('Unknown', inplace=True)

In [14]:
jacobs_ethnicities['year'] = jacobs_ethnicities['year.name'].str.split(" ", expand=True)[0]
jacobs_ethnicities = jacobs_ethnicities.rename({'ethnic.l1': "Ethnicity"}, axis=1)
#jacobs_ethnicities

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours,Ethnicity,year
0,2015 Fall,1474600,Undergraduate,Engineering Science,Design Innovation,90,Primary,1,18400,101548.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,2.0,International,2015
1,2015 Fall,2631969,Undergraduate,Engineering Science,Design Innovation,90,Primary,1,18400,101548.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,2.0,Asian/Pacific Islander,2015
2,2015 Spring,723198,Graduate,Engineering Science,Design Innovation,190,Primary,2,18406,18406.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Unknown,2015
3,2015 Spring,917618,Undergraduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Underrepresented Minority,2015
4,2015 Spring,1025236,Graduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Unknown,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,2019 Summer,4262284,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,A,Letter Grade,A-,3.7,2.0,Unknown,2019
2251,2019 Summer,4262621,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,A,Letter Grade,A-,3.7,2.0,Unknown,2019
2252,2019 Summer,4264306,Undergraduate,Design Innovation,Design Innovation,190,Primary,8,15779,15779.0,Spec Topics Des Inv,A,Letter Grade,A+,4.0,3.0,Unknown,2019
2253,2019 Summer,4265873,Undergraduate,Design Innovation,Design Innovation,190,Primary,7,11577,11577.0,Spec Topics Des Inv,B,Letter Grade,B,3.0,2.0,Unknown,2019
